In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
physical_devices = tf.config.list_physical_devices('GPU')
sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())

                    ndatetime   open   high    low  close  volume
10137 2021-11-08 09:32:28.900  17308  17348  17306  17307   12000
10138 2021-11-08 10:07:25.337  17307  17381  17305  17360   12000
10139 2021-11-08 10:38:16.024  17360  17385  17342  17381   12000
10140 2021-11-08 12:16:31.856  17381  17417  17366  17412   12000
10141 2021-11-08 13:33:00.831  17412  17425  17411  17424    5214


In [9]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm

In [10]:
Candledf['Y'] = Candledf.close.shift(-1)


In [11]:
def train_windows(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,1:5]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["Y"]))
    return np.array(X_train), np.array(Y_train)

In [12]:
stock = Candledf[:Candledf.shape[0]-613]
test = Candledf[-612:]
test = test.reset_index(drop=True)
stock = stock.reset_index(drop=True)

In [13]:
stock = stock[['open','high','low','close','Y']]
test = test[['open','high','low','close','Y']]
stock.tail()

,open,high,low,close,Y
9524,17475,17477,17462,17473,17473.0
9525,17474,17474,17472,17473,17465.0
9526,17458,17496,17453,17465,17431.0
9527,17466,17468,17418,17431,17380.0
9528,17431,17434,17380,17380,17458.0


In [14]:
X_train, Y_train = train_windows(stock, 12, 1)
X_test, Y_test = train_windows(test, 12, 1)
print(X_train.shape)
print(Y_train.shape)

(9517, 12, 4)
(9517, 1)


In [23]:
class stock_lstm(tf.keras.Model):
  def __init__(self,units):
      super(stock_lstm, self).__init__()
      self.rnn = tf.keras.Sequential([tf.keras.layers.LSTM(units,input_shape=(12,4),dropout=0.2)])
      self.fc = tf.keras.layers.Dense(50)
      self.out = tf.keras.layers.Dense(1)
  def call(self, inputs, training=None):
      x = self.rnn(inputs)
      x = self.fc(x)
      out = self.out(x)
      return out

In [24]:
batchsz = 12

data = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
data = data.batch(batchsz,drop_remainder=True)
# list(data.as_numpy_iterator())

data_test = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
data_test = data_test.batch(batchsz,drop_remainder=True)
# list(data_test.as_numpy_iterator())


In [25]:
units = 50
epochs = 10


rnn_model = stock_lstm(units)
rnn_model.compile(optimizer  =tf.keras.optimizers.Adam(),loss="mean_absolute_error",metrics=['mean_absolute_error'])
rnn_model.fit(data,epochs=epochs,validation_data=data_test,batch_size=12)

Epoch 1/10


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[stock_lstm_4/sequential_4/lstm_6/PartitionedCall]] [Op:__inference_train_function_14027]

Function call stack:
train_function -> train_function -> train_function
